In [2]:
import numpy as np
from skimage.segmentation import slic
import scipy.ndimage
import scipy.spatial
import torch
from torchvision import datasets
from torchvision import datasets
from chebygin import ChebyGIN
from extract_superpixels import process_image
from graphdata import comput_adjacency_matrix_images
from utils import list_to_torch, data_to_device

# MNIST-75sp

In [3]:
data_dir = './data'
checkpoints_dir = './checkpoints'

In [4]:
state = torch.load('%s/checkpoint_mnist-75sp_ws13.cfs.uoguelph.ca_401193_epoch2_seed0000011.pth.tar' % checkpoints_dir)
args = state['args']
model = ChebyGIN(in_features=5,
                 out_features=10,
                 filters=args.filters,
                 K=args.filter_scale,
                 n_hidden=args.n_hidden,
                 aggregation=args.aggregation,
                 dropout=args.dropout,
                 readout=args.readout,
                 pool=args.pool,
                 pool_arch=args.pool_arch)
#print(model)
model.load_state_dict(state['state_dict'])
model.eval()

ChebyGINLayer torch.Size([4, 20]) tensor([0.5234, 0.5761, 0.6192, 0.5504], grad_fn=<SliceBackward>)
ChebyGINLayer torch.Size([64, 16]) tensor([0.4932, 0.6397, 0.5588, 0.4526, 0.5127, 0.6370, 0.5917, 0.5374, 0.5728,
        0.5277], grad_fn=<SliceBackward>)
ChebyGINLayer torch.Size([512, 256]) tensor([0.5578, 0.5937, 0.5798, 0.5754, 0.5991, 0.5613, 0.5451, 0.5795, 0.6196,
        0.5481], grad_fn=<SliceBackward>)


ChebyGIN(
  (graph_layers): Sequential(
    (0): ChebyGINLayer(in_features=5, out_features=4, K=4, n_hidden=0, aggregation=mean)
    fc=Sequential(
      (0): Linear(in_features=20, out_features=4, bias=True)
      (1): ReLU(inplace)
    )
    (1): ChebyGINLayer(in_features=4, out_features=64, K=4, n_hidden=0, aggregation=mean)
    fc=Sequential(
      (0): Linear(in_features=16, out_features=64, bias=True)
      (1): ReLU(inplace)
    )
    (2): ChebyGINLayer(in_features=64, out_features=512, K=4, n_hidden=0, aggregation=mean)
    fc=Sequential(
      (0): Linear(in_features=256, out_features=512, bias=True)
      (1): ReLU(inplace)
    )
    (3): GraphReadout(max)
  )
  (fc): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [6]:
data = datasets.MNIST(data_dir, train=False, download=True)
images = data.test_data.numpy()
targets = data.test_labels
mn = torch.tensor([0.11221933, 0.11221933, 0.11221933, 0.44187942, 0.43931073]).view(1, 1, -1)
sd = torch.tensor([0.27220142, 0.27220142, 0.27220142, 0.29885307, 0.3009136]).view(1, 1, -1)
args.n_sp = 75
args.compactness = 0.25
args.split = 'test'
args.dataset = 'mnist'
img_size = images.shape[1]
n_images = 10000
pred = []
for i in range(n_images):
    sp_intensity, sp_coord, sp_order, superpixels = process_image((images[i], i, n_images, args))
    sp_coord = sp_coord / img_size
    x = torch.from_numpy(np.pad(np.concatenate((sp_intensity, sp_coord), axis=1), 
                                ((0, 0), (2, 0)), 'edge')).unsqueeze(0)
    N_nodes = x.shape[0]
    A = torch.from_numpy(comput_adjacency_matrix_images(sp_coord)).float().unsqueeze(0)
    y = model([(x - mn) / sd, A, torch.ones(1, N_nodes), -1, {'N_nodes': N_nodes}])[0]
    pred.append(torch.argmax(y))
    print(torch.mean((torch.stack(pred) == targets[:len(pred)]).float()).item() * 100)

/export/mlrg/bknyazev/anaconda2/envs/py3/lib/python3.6/site-packages/skimage/segmentation/slic_superpixels.py:156: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  segments_z = grid_z[slices]
/export/mlrg/bknyazev/anaconda2/envs/py3/lib/python3.6/site-packages/skimage/segmentation/slic_superpixels.py:157: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  segments_y = grid_y[slices]
/export/mlrg/bknyazev/anaconda2/envs/py3/lib/python3.6/site-packages/skimage/segmentation/slic_superpixels.py:158: FutureWarning: Using a non-tuple sequence for multi

image=0/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=74
100.0
image=1/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=71
100.0
image=2/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=46
100.0
image=3/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=66
100.0
image=4/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=75
100.0
image=5/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=75
100.0
image=6/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=75
100.0
image=7/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=72
100.0
image=8/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=69
88.88888955116272
image=9/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=68
89.99999761581421
image=10/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=71
90.90909361839294
image=11/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=71
91.66666865348816
image=12/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=73
92.30769276618958
image=13/10000, shape=(28, 28, 1), min=0.00, max=1.00

image=110/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=71
95.4954981803894
image=111/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=45
95.53571343421936
image=112/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=73
95.57521939277649
image=113/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=75
95.61403393745422
image=114/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=75
95.652174949646
image=115/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=46
95.68965435028076
image=116/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=46
95.72649598121643
image=117/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=72
95.76271176338196
image=118/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=73
95.79831957817078
image=119/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=70
95.83333134651184
image=120/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=71
95.86777091026306
image=121/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=74
95.90163826942444
image=122/10000, sh

94.47004795074463
image=217/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=74
94.03669834136963
image=218/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=72
93.60730648040771
image=219/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=72
93.63636374473572
image=220/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=71
93.66515874862671
image=221/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=71
93.69369149208069
image=222/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=67
93.72197389602661
image=223/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=72
93.75
image=224/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=75
93.77777576446533
image=225/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=73
93.80530714988708
image=226/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=72
93.83260011672974
image=227/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=74
93.85964870452881
image=228/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=46
93.88646483421326
image=229/

image=324/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=75
92.92307496070862
image=325/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=71
92.94478297233582
image=326/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=46
92.9663598537445
image=327/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=68
92.98780560493469
image=328/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=73
93.00912022590637
image=329/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=75
93.03030371665955
image=330/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=75
93.05136203765869
image=331/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=74
93.07228922843933
image=332/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=74
93.09309124946594
image=333/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=72
93.113774061203
image=334/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=72
93.13432574272156
image=335/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=75
93.15476417541504
image=336/10000, sh

...

92.00336933135986
image=9504/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=72
92.0042097568512
image=9505/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=72
92.00505018234253
image=9506/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=70
92.00589060783386
image=9507/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=75
92.0067310333252
image=9508/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=72
92.00757145881653
image=9509/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=73
92.00841188430786
image=9510/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=70
92.0092523097992
image=9511/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=73
92.01009273529053
image=9512/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=70
92.01093316078186
image=9513/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=75
92.0117735862732
image=9514/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=75
92.01261401176453
image=9515/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=73
92.01344

92.04829335212708
image=9608/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=72
92.03871488571167
image=9609/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=68
92.03954339027405
image=9610/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=74
92.02996492385864
image=9611/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=69
92.03079342842102
image=9612/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=74
92.0316219329834
image=9613/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=72
92.03245043754578
image=9614/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=70
92.03327894210815
image=9615/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=69
92.03410744667053
image=9616/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=73
92.03493595123291
image=9617/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=66
92.03576445579529
image=9618/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=72
92.03659296035767
image=9619/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=69
92.03

92.0391321182251
image=9710/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=67
92.039954662323
image=9711/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=72
92.03047752380371
image=9712/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=71
92.03130006790161
image=9713/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=73
92.03211665153503
image=9714/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=70
92.03293919563293
image=9715/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=75
92.03375577926636
image=9716/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=69
92.02428460121155
image=9717/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=72
92.02510714530945
image=9718/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=74
92.02592968940735
image=9719/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=72
92.01645851135254
image=9720/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=68
92.01728105545044
image=9721/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=71
92.0078

91.85773730278015
image=9813/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=75
91.858571767807
image=9814/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=74
91.85940027236938
image=9815/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=73
91.86022877693176
image=9816/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=75
91.86105728149414
image=9817/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=75
91.86188578605652
image=9818/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=69
91.8627142906189
image=9819/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=73
91.86354279518127
image=9820/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=71
91.86437129974365
image=9821/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=68
91.86519980430603
image=9822/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=74
91.86602830886841
image=9823/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=73
91.86685681343079
image=9824/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=73
91.8676

91.79782271385193
image=9912/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=74
91.7986512184143
image=9913/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=71
91.7994737625122
image=9914/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=74
91.80030226707458
image=9915/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=74
91.80113077163696
image=9916/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=73
91.80195331573486
image=9917/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=70
91.80278182029724
image=9918/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=73
91.80361032485962
image=9919/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=73
91.80443286895752
image=9920/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=72
91.8052613735199
image=9921/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=72
91.80608987808228
image=9922/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=74
91.80691242218018
image=9923/10000, shape=(28, 28, 1), min=0.00, max=1.00, n_sp=72
91.8077